In [95]:
%pip install datasets

from datasets import load_dataset
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from tqdm import tqdm


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to /Users/beast-sl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [113]:
dataset = load_dataset("cc_news", split="train")
dataset = dataset.to_pandas()[['text', 'domain', 'date']].replace(r'^\s*$', np.nan, regex=True).dropna()

tqdm.pandas(desc="news dataset")
dataset['tokenized_text'] = dataset['text'].progress_apply(sent_tokenize)

dataset['length'] = dataset['tokenized_text'].apply(lambda x: len(x))
dataset = dataset.drop(index=dataset[dataset['length'] < 5].index)
dataset = dataset.sort_values(by=['text', 'domain'])
dataset = dataset.drop_duplicates(subset=['text'])
dataset = dataset.sort_values(['date', 'domain'], ascending=False)

sources = dataset['domain'].value_counts()
sources = sources[sources > 100]
dataset = dataset[dataset['domain'].isin(sources.index)]

dataset['aggregated_length'] = dataset['length'].cumsum()
dataset = dataset[dataset['aggregated_length'] < 20000]
dataset['paper_id'] = dataset.index
dataset

Found cached dataset cc_news (/Users/beast-sl/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/ae469e556251e6e7e20a789f93803c7de19d0c4311b6854ab072fecb4e401bd6)
news dataset: 100%|██████████| 630643/630643 [03:32<00:00, 2967.60it/s] 


,text,domain,date,tokenized_text,length,aggregated_length,paper_id
421969,"The leaders of Greece, Bulgaria, Romania and S...",www.ekathimerini.com,2018-07-25 00:00:00,"[The leaders of Greece, Bulgaria, Romania and ...",9,9,421969
593236,type Movie genre Horror release date 07/04/18 ...,ew.com,2018-07-25 00:00:00,[type Movie genre Horror release date 07/04/18...,41,50,593236
422071,"On any given day, in a sleepy rural village ne...",www.ekathimerini.com,2018-07-21 00:00:00,"[On any given day, in a sleepy rural village n...",39,89,422071
200071,"China will ""absolutely not"" fire the first sho...",www.news18.com,2018-07-20 00:00:00,"[China will ""absolutely not"" fire the first sh...",15,104,200071
484538,Hundreds of thousands of Americans took to the...,www.motherjones.com,2018-07-20 00:00:00,[Hundreds of thousands of Americans took to th...,59,163,484538
...,...,...,...,...,...,...,...
260639,This week Helen McGinn (pictured) tries Les Et...,www.dailymail.co.uk,2018-07-04 22:47:48,[This week Helen McGinn (pictured) tries Les E...,19,19915,260639
264064,Her billionaire ex-husband James Packer was re...,www.dailymail.co.uk,2018-07-04 22:47:19,[Her billionaire ex-husband James Packer was r...,18,19933,264064
48398,Follow @insidefutbol\nChelsea have touched bas...,www.insidefutbol.com,2018-07-04 22:47:10,[Follow @insidefutbol\nChelsea have touched ba...,8,19941,48398
635235,MADRID: Cristiano Ronaldo has received an offe...,economictimes.indiatimes.com,2018-07-04 22:47:00,[MADRID: Cristiano Ronaldo has received an off...,6,19947,635235


In [116]:
dataset.drop(['aggregated_length', 'tokenized_text'], axis=1).to_csv('raw_dataset.csv', index=False)

In [117]:
tokenized_dataset = dataset.explode('tokenized_text') \
             .drop(['text', 'length', 'aggregated_length'], axis=1) \
             .rename(columns={'tokenized_text': 'text'}) \
             .reset_index()
tokenized_dataset['sentence_id'] = tokenized_dataset.index
tokenized_dataset.to_csv('sent_tokenized_dataset.csv', index=False)